In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Basic configuration
DATE_CONFIG = {
    'TRAIN_START': pd.Timestamp('2021-02-02'),
    'TRAIN_END': pd.Timestamp('2024-01-01'),
    'TEST_END': pd.Timestamp('2025-01-01'),
    'TRADING_DAYS_PER_YEAR': 252  
}

def get_training_period():
    return {
        'start': DATE_CONFIG['TRAIN_START'],
        'end': DATE_CONFIG['TRAIN_END']
    }

def get_test_period():
    return {
        'start': DATE_CONFIG['TRAIN_END'],
        'end': DATE_CONFIG['TEST_END']
    }

def get_training_days():
    years = (DATE_CONFIG['TRAIN_END'] - DATE_CONFIG['TRAIN_START']).days / 365
    return int(years * DATE_CONFIG['TRADING_DAYS_PER_YEAR'])

# Plot settings
plt.style.use('classic')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

In [ ]:
def load_and_prepare_data(file_path):

    df = pd.read_parquet(file_path)
    df['date'] = pd.to_datetime(df['date'])
    
    mask = (df['date'] >= DATE_CONFIG['TRAIN_START']) & \
           (df['date'] <= DATE_CONFIG['TEST_END'])
    df = df[mask]
    
    price_matrix = df.pivot(index='date', columns='symbol', values='close')
    
    symbols = price_matrix.columns.tolist()
    
    print(f"Loaded data from {DATE_CONFIG['TRAIN_START']} to {DATE_CONFIG['TEST_END']}")
    print(f"Total symbols: {len(symbols)}")
    print(f"Total trading days: {len(price_matrix)}")
    
    return price_matrix, symbols

In [ ]:
def find_cointegrated_pairs(price_matrix, pvalue_threshold=0.03):
    n = price_matrix.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    symbols = price_matrix.columns
    pairs = []
    results = []
    
    total_pairs = sum(range(n))
    with tqdm(total=total_pairs, desc="Analyzing pairs") as pbar:
        for i in range(n):
            for j in range(i+1, n):
                S1 = price_matrix[symbols[i]]
                S2 = price_matrix[symbols[j]]
                
                result = coint(S1, S2)
                score = result[0]
                pvalue = result[1]
                
                score_matrix[i, j] = score
                pvalue_matrix[i, j] = pvalue
                
                results.append({
                    'symbol1': symbols[i],
                    'symbol2': symbols[j],
                    'p_value': pvalue,
                    'score': score
                })
                
                if pvalue <= pvalue_threshold:
                    pairs.append((symbols[i], symbols[j]))
                    
                pbar.update(1)
    
    return score_matrix, pvalue_matrix, pairs, results

def analyze_pairs(price_matrix, pvalue_threshold=0.03):
    """
    Runs the cointegration analysis and prints summary
    """
    score_matrix, pvalue_matrix, pairs, results = find_cointegrated_pairs(
        price_matrix, 
        pvalue_threshold
    )
    
    print(f"\nAnalysis complete!")
    print(f"Found {len(pairs)} cointegrated pairs")
    print(f"Total pairs analyzed: {len(results)}")
    
    summary_df = pd.DataFrame(results)
    summary_df['is_cointegrated'] = summary_df['p_value'] <= pvalue_threshold
    
    return score_matrix, pvalue_matrix, pairs, summary_df

def plot_cointegration_heatmap(pvalue_matrix, symbols, max_pvalue=0.98):
    """
    Creates a heatmap visualization of the cointegration p-values
    """
    plt.figure(figsize=(12, 8))
    mask = (pvalue_matrix >= max_pvalue)
    
    sns.heatmap(
        pvalue_matrix, 
        xticklabels=symbols, 
        yticklabels=symbols, 
        cmap='RdYlGn_r',
        mask=mask
    )
    
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.title('Cointegration p-values heatmap')
    plt.tight_layout()
    plt.show()

In [ ]:
price_matrix, symbols = load_and_prepare_data('./nasdaq_daily.parquet')

score_matrix, pvalue_matrix, pairs, summary_df = analyze_pairs(
    price_matrix,
    pvalue_threshold=0.03
)

plt.figure(figsize=(10, 5))
plt.hist(summary_df['p_value'], bins=50)
plt.title('Distribution of Cointegration p-values')
plt.xlabel('p-value')
plt.ylabel('Frequency')
plt.show()

plot_cointegration_heatmap(pvalue_matrix, symbols)

print("\nTop 10 most cointegrated pairs (lowest p-values):")
top_pairs = summary_df.nsmallest(10, 'p_value')
print(top_pairs[['symbol1', 'symbol2', 'p_value']].to_string())

print(f"\nSummary Statistics:")
print(f"Total pairs analyzed: {len(summary_df)}")
print(f"Cointegrated pairs (p <= 0.01): {summary_df['is_cointegrated'].sum()}")
print(f"Average p-value: {summary_df['p_value'].mean():.4f}")
print(f"Median p-value: {summary_df['p_value'].median():.4f}")

In [ ]:
def zscore(series):
    return (series - series.mean()) / np.std(series)

def calculate_spread(data, symbol1, symbol2, start_date=None, end_date=None):
    if start_date:
        mask = (data.index >= start_date) & (data.index <= end_date)
        data = data[mask]
    
    # Calculate ratio and z-score
    ratios = data[symbol1] / data[symbol2]
    zscore_ratios = zscore(ratios)
    
    return ratios, zscore_ratios

def trade(S1_train, S2_train, S1_test, S2_test, symbol1, symbol2, window1=5, window2=100):
    ratios_train = S1_train / S2_train
    ma2_train = ratios_train.rolling(window=window2, center=False).mean()
    std_train = ratios_train.rolling(window=window2, center=False).std()
    
    ratios_test = S1_test / S2_test
    trades = []
    trade_id = 0
    position = 0
    entry_prices = None
    entry_date = None
    open_trades = 0
    
    for i in range(len(ratios_test)):
        current_ratio = ratios_test.iloc[i]
        current_date = ratios_test.index[i]
        
        ma2_test = ratios_test.iloc[:i+1].rolling(window=window2, center=False).mean().iloc[-1]
        std_test = ratios_test.iloc[:i+1].rolling(window=window2, center=False).std().iloc[-1]
        zscore = (current_ratio - ma2_test) / std_test
        
        if position == 0:
            if zscore > 2.0:
                entry_date = current_date
                entry_prices = {
                    symbol1: {"price": S1_test.iloc[i], "type": "short"},
                    symbol2: {"price": S2_test.iloc[i], "type": "long"}
                }
                position = -1
                
            elif zscore < -2.0:
                entry_date = current_date
                entry_prices = {
                    symbol1: {"price": S1_test.iloc[i], "type": "long"},
                    symbol2: {"price": S2_test.iloc[i], "type": "short"}
                }
                position = 1
                
        elif (position == -1 and zscore > 4) or (position == 1 and zscore < -4) or abs(zscore) < 0.5:
            for symbol in [symbol1, symbol2]:
                trades.append({
                    'trade_id': trade_id,
                    'symbol': symbol,
                    'entry_date': entry_date,
                    'entry_price': entry_prices[symbol]["price"],
                    'exit_date': current_date,
                    'exit_price': S1_test.iloc[i] if symbol == symbol1 else S2_test.iloc[i],
                    'position_type': entry_prices[symbol]["type"],
                    'paired_symbol': symbol2 if symbol == symbol1 else symbol1,
                    'exit_type': 'stop_loss' if ((position == -1 and zscore > 2.0) or (position == 1 and zscore < -2.0)) else 'target'
                })
            position = 0
            trade_id += 1
    
    if position != 0:
        open_trades += 1
            
    return trades, open_trades


def backtest_pairs(price_matrix, pairs, train_end_date):
    all_trades = []
    total_open_trades = 0
    
    for symbol1, symbol2 in pairs:
        training_mask = price_matrix.index < train_end_date
        
        S1_train = price_matrix[symbol1][training_mask]
        S2_train = price_matrix[symbol2][training_mask]
        S1_test = price_matrix[symbol1][~training_mask]
        S2_test = price_matrix[symbol2][~training_mask]
        
        pair_trades, open_trades = trade(S1_train, S2_train, S1_test, S2_test, symbol1, symbol2)
        all_trades.extend(pair_trades)
        total_open_trades += open_trades
    
    trades_df = pd.DataFrame(all_trades)
    trades_df.to_parquet('./results/cointegration_zscore_results.parquet')
    
    print(f"Total open trades at end: {total_open_trades}")
    
    return trades_df

In [32]:
trades_df = backtest_pairs(price_matrix, pairs, DATE_CONFIG['TRAIN_END'])

Total open trades at end: 93


In [ ]:
print("Gefundene Pairs:")
for pair in pairs:
    print(f"{pair[0]} - {pair[1]}")
print(f"\nGesamtanzahl der Pairs: {len(pairs)}")